In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.metrics import precision_score, recall_score

# 날짜 파싱 함수 예시
def parse_date(date_str):
    """
    날짜 컬럼이 공백이거나 잘못된 포맷일 경우 None 반환
    날짜 포맷은 예: '9/6/2022', '2022-09-06' 등 다양할 수 있으니
    상황에 맞게 처리. 여기서는 간단히 시도.
    """
    if pd.isna(date_str) or date_str=="":
        return None
    for fmt in ("%m/%d/%Y", "%Y-%m-%d", "%m/%d/%Y %H:%M:%S"):  # 필요한 포맷 시도
        try:
            return datetime.strptime(date_str, fmt)
        except:
            pass
    return None

# 예시: 이미 주어진 "의도적 조기상환 꾼" 데이터셋 (df_kkun) 과
#       "꾼이 아닌 사람" 데이터셋 (df_not_kkun) 이 있다고 가정.
# 실제로는 pd.read_csv("...") 형태로 로드하면 됩니다.

df_kkun = pd.read_csv("꾼.csv")
df_not_kkun = pd.read_csv("꾼아님.csv")

# 날짜 컬럼들 datetime 변환
for df in [df_kkun, df_not_kkun]:
    df["openedAt"] = df["openedAt"].apply(parse_date)
    df["dueAt"] = df["dueAt"].apply(parse_date)
    df["lastRepaymentAt"] = df["lastRepaymentAt"].apply(parse_date)


In [2]:
def is_quick_repayment(row, ratio_threshold=0.5):
    """
    (lastRepaymentAt - openedAt) / (dueAt - openedAt) < ratio_threshold 이면
    '빠른 상환'으로 간주.
    dueAt, lastRepaymentAt이 None이면 False 처리(또는 제외).
    """
    if (row["openedAt"] is None) or (row["dueAt"] is None) or (row["lastRepaymentAt"] is None):
        return False
    
    total_term = (row["dueAt"] - row["openedAt"]).days
    actual_term = (row["lastRepaymentAt"] - row["openedAt"]).days
    if total_term <= 0:
        return False  # dueAt <= openedAt 인 경우도 예외처리
    
    return (actual_term < ratio_threshold * total_term)

def classify_user_as_kkun(df_single_user):
    """
    특정 user의 모든 대출 정보를 받아,
    1) '빠른 상환' 대출이 3건 이상 존재하는지,
    2) 그 후 큰 금액 lost(부실) 대출이 있는지
    를 체크하여 True/False 리턴
    """
    # 1) 우선 '빠른 상환'인 loan만 필터링
    df_single_user["quick"] = df_single_user.apply(is_quick_repayment, axis=1)
    quick_loans = df_single_user[df_single_user["quick"] == True]
    
    # 2) quick loanAmount 평균
    avg_quick_loan = quick_loans["loanAmount"].mean() if len(quick_loans)>0 else 0
    
    # 3) lost(부실) 상태 중에서, loanAmount >= 2 * avg_quick_loan 인 대출이 있는지
    #    그리고 quick loan이 3건 이상인지
    if (len(quick_loans) >= 3) and (avg_quick_loan > 0):
        # lost 중에서 '충분히 큰 금액' 조건을 만족하는게 하나라도 있으면 True
        lost_loans = df_single_user[df_single_user["status"]=="lost"]
        for _, row in lost_loans.iterrows():
            if row["loanAmount"] >= 2 * avg_quick_loan:
                return True
    
    return False

def get_predictions(df):
    """
    DataFrame을 userId별로 그룹핑하여 classify_user_as_kkun 함수를 적용.
    user별 결과(True/False) -> 다시 df와 merge하여 각 row마다 동일한 예측 라벨 반환.
    """
    # user별로 분류 결과를 dict로 저장
    user_labels = {}
    for user_id, group in df.groupby("userId"):
        user_labels[user_id] = classify_user_as_kkun(group)
    
    # df 순회하며 해당 userId의 분류 라벨을 달아줌
    preds = df["userId"].apply(lambda x: 1 if user_labels[x] else 0)
    return preds


In [3]:
# 1) 라벨링(실제 값)
y_true_kkun = [1]*len(df_kkun)       # 의도적 조기상환 꾼
y_true_not = [0]*len(df_not_kkun)    # 꾼 아님

# 2) 데이터 concat
df_all = pd.concat([df_kkun, df_not_kkun], ignore_index=True)
y_true = y_true_kkun + y_true_not

# 3) 예측
y_pred = get_predictions(df_all)  # 위에서 만든 함수

# 4) 정밀도, 재현율
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)

print("정밀도(Precision):", precision)
print("재현율(Recall):", recall)


정밀도(Precision): 0.4202247191011236
재현율(Recall): 0.8008565310492506
